In [2]:
import pandas as pd
import sklearn.metrics as metr
import numpy as np
#from ignite.metrics import Precision
import torch
from torchvision import models
import os
import time

In [86]:
m = models.resnet50(pretrained=True)
for param in m.parameters():
    param.requires_grad = False
input_num = 3
out_shape = 2
m.conv1 = torch.nn.Conv2d(input_num, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
last_layer_in_channels = m.fc.in_features
m.fc = torch.nn.Linear(last_layer_in_channels, out_shape)

In [87]:
device = 'cuda'
model_path = '../Saved_models/'
model_name = 'Saved_models'
model = m
model.load_state_dict(torch.load(model_path+model_name))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [81]:
class Metrics():
    
    def __init__(self):
        
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0
        
    def update(self, y_true, y_pred):
        tn, fp, fn, tp = metr.confusion_matrix(y_true, y_pred).ravel()
        self.tp += tp
        self.fp += fp
        self.fn += fn
        self.tn += tn
        
    def compute(self):
        acc = (self.tp + self.tn) / (self.tp + self.tn + self.fp + self.fn) # TP + TN / (TP + TN + FP + FN)
        prec = self.tp / (self.tp + self.fp)                                # TP / (TP + FP)
        rec = self.tp / (self.tp + self.fn)                                 # TP / (TP + FN)
        f1 = 2 * prec * rec / (rec + prec)                                  # 2 * PREC * REC / (PREC + REC)
        return acc, prec, rec, f1
    
    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0
        

In [83]:
M = Metrics()
M.update(y.ravel(), y_pred.ravel())
print(M.compute())
M.reset()
print(M.tp, M.fp)

(0.75, 0.5, 1.0, 0.6666666666666666)
0 0


In [97]:

#### здесь как предсказывал ignite
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')
metric = Metrics()
args = {
    'image_path': '../Data/h5/',
    'attribute':['attribute_pigment_network', 'attribute_milia_like_cyst'],
    'mask_use':False,
    'batch_size': 2
}
loader = make_loader(train_test_id, mask_ind, args, train='valid', shuffle=True)
for i, (image_batch, labels_batch, names) in enumerate(loader):
    image_batch = image_batch.permute(0, 3, 1, 2).to(device).type(torch.cuda.FloatTensor)
    labels_batch = labels_batch.to(device).type(torch.cuda.FloatTensor)
    output_probs = model(image_batch)
    outputs = torch.sigmoid(output_probs)

    print(labels_batch.data.cpu().numpy().ravel(), np.around(outputs.data.cpu().numpy().ravel()))
    
    metric.update(labels_batch.data.cpu().numpy().ravel(), np.around(outputs.data.cpu().numpy().ravel()))
    metrics = metric.compute()
    print('Acc: {:.4f} Prec: {:.4f} Recall: {:.4f}'.format(
            metrics[0], metrics[1], metrics[2]))
metric.reset()

[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.8750 Prec: 0.7500 Recall: 1.0000
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.8500 Prec: 0.8000 Recall: 0.8889
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.8333 Prec: 0.7500 Recall: 0.9000
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.8214 Prec: 0.7143 Recall: 0.9091
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.6250 Recall: 0.8333
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7778 Prec: 0.6667 Recall: 0.8571
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.6000 Recall: 0.8571
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.5909 Recall: 0.8667
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7083 Prec: 0.5417 Recall: 0.8125
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.7115 Prec: 0.5769 Recall: 0.7895
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7143 Prec: 0.5714 Recall: 0.8000
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7167 Prec: 0.

[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6789 Prec: 0.5894 Recall: 0.7178
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6794 Prec: 0.5887 Recall: 0.7192
[0. 1. 1. 0.] [0. 1. 0. 1.]
Acc: 0.6780 Prec: 0.5880 Recall: 0.7171
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6786 Prec: 0.5873 Recall: 0.7184
[1. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6791 Prec: 0.5906 Recall: 0.7177
[0. 1. 1. 0.] [0. 1. 0. 1.]
Acc: 0.6777 Prec: 0.5898 Recall: 0.7156
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6764 Prec: 0.5853 Recall: 0.7156
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6731 Prec: 0.5808 Recall: 0.7123
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6756 Prec: 0.5840 Recall: 0.7150
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6780 Prec: 0.5871 Recall: 0.7176
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.6786 Prec: 0.5902 Recall: 0.7169
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6791 Prec: 0.5896 Recall: 0.7182
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6796 Prec: 0.5889 Recall: 0.7195
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6801 Prec: 0.5882 Recall: 0.7207
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6807 Prec: 0.

# Data

In [13]:
path = '../Data/'
tti_name = 'train_test_id.csv'
mi_name = 'mask_ind.csv'
os.listdir('../Data/')

['h5', 'train_test_id.csv', 'skin_images', 'mask_ind.csv', 'skin_masks']

In [11]:
tti = pd.read_csv(path+tti_name)
tti.head()

,ID,Split
0,ISIC_0000000,train
1,ISIC_0000001,train
2,ISIC_0000003,train
3,ISIC_0000004,train
4,ISIC_0000006,train


In [14]:
mi = pd.read_csv(path+mi_name)
mi.head()

,globules,milia_like_cyst,negative_network,pigment_network,streaks
0,0,0,0,1,0
1,1,0,0,1,1
2,0,1,0,1,0
3,0,0,0,0,0
4,0,0,0,1,0


# Test step

In [2]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../')
from train import make_step
from models import create_model
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.backends import cudnn
import torch
from metrics import Metrics
from loss import LossBinary

Using TensorFlow backend.


ImportError: cannot import name 'LossBinary'

In [ ]:
class Args:
    
    def __init__(self):
        self.attribute = ['attribute_milia_like_cyst', 'attribute_pigment_network']
        self.pretrained = True
        self.jaccard_weight = 0.
        self.show_model = True
        self.mask_use = True
        self.freezing = True
        self.prob = 0.1
        self.image_path = '../Data/h5/'
        self.batch_size = 2
        self.model = 'resnet50'
        self.lr = 0.00001
        self.augment_list = []
        self.N = 0
device = 'cuda'

In [53]:
args = Args()
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')

In [54]:
cudnn.benchmark = True
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model, optimizer = create_model(args, device)
metric = Metrics()
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=10, verbose=True)
criterion = LossBinary(args.jaccard_weight)
metric = Metrics()
print(model)

ResNet(
  (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [55]:
mode = 'train'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.168882 Accuracy: 0.8636 Precision: 0.8715 Recall: 0.8940 F1: 0.8826 Time: 113.1284


In [56]:
mode = 'valid'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.683116 Accuracy: 0.6617 Precision: 0.7229 Recall: 0.6838 F1: 0.7028 Time: 5.5147


In [4]:
mask = np.array([[0,1,2],
                 [3,4,5],
                 [6,7,8]])
mask[2:,2:].fill(0.)
print(mask)

[[0 1 2]
 [3 4 5]
 [6 7 0]]


# Test step

In [7]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../')
from train import make_step
from models import create_model
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.backends import cudnn
import torch
from metrics import Metrics
from loss import LossBinary

ImportError: cannot import name 'LossBinary'

In [52]:
class Args:
    
    def __init__(self):
        self.attribute = ['attribute_milia_like_cyst', 'attribute_pigment_network']
        self.pretrained = True
        self.jaccard_weight = 0.
        self.show_model = True
        self.mask_use = True
        self.freezing = True
        self.prob = 0.1
        self.image_path = '../Data/h5/'
        self.batch_size = 2
        self.model = 'resnet50'
        self.lr = 0.00001
        self.augment_list = []
        self.N = 0
device = 'cuda'

In [53]:
args = Args()
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')

In [54]:
cudnn.benchmark = True
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model, optimizer = create_model(args, device)
metric = Metrics()
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=10, verbose=True)
criterion = LossBinary(args.jaccard_weight)
metric = Metrics()
print(model)

ResNet(
  (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [55]:
mode = 'train'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.168882 Accuracy: 0.8636 Precision: 0.8715 Recall: 0.8940 F1: 0.8826 Time: 113.1284


In [56]:
mode = 'valid'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.683116 Accuracy: 0.6617 Precision: 0.7229 Recall: 0.6838 F1: 0.7028 Time: 5.5147


In [3]:
MSE = torch.nn.MSELoss()
t1 = torch.Tensor([1,2,3])
t2 = torch.Tensor([1.2,2.2,3.2])
res = MSE(t1,t2)
print(res)

tensor(0.0400)


In [5]:
MSE2 = torch.nn.MSELoss(reduction='sum')
res2 = MSE2(t1,t2)
print(res2)

tensor(0.1200)


# Loader

In [2]:
import os
import torchvision.transforms.functional as TF
from keras.preprocessing.image import array_to_img, img_to_array
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import torch
import h5py
from torch.optim.lr_scheduler import ReduceLROnPlateau


def load_image(file_name, type='image'):
    f = h5py.File(file_name, 'r')
    file_np = f['img'][()]
    if type == 'image':
        file_np = (file_np / 255.0).astype('float32')
    elif type == 'mask':
        file_np = file_np.astype('uint8')
    else:
        print('not choosed type to load')
        return
    return file_np


class MyDataset(Dataset):

    def __init__(self, train_test_id, labels_ids, args, train):

        self.train_test_id = train_test_id
        self.labels_ids = labels_ids
        self.image_path = args.image_path
        self.pretrained = args.pretrained
        self.attribute = args.attribute
        self.mask_use = args.mask_use
        self.augment_list = args.augment_list
        self.prob = args.prob
        self.train = train
        self.all_attributes = ['attribute_globules', 'attribute_milia_like_cyst', 'attribute_negative_network',
                               'attribute_pigment_network', 'attribute_streaks']

        self.indexes = [i for i, val in enumerate(self.all_attributes) for attr in self.attribute if attr == val]
        self.cell = args.cell
        self.cell_size = args.cell_size

        if train == 'train':
            self.labels_ids = self.labels_ids[self.train_test_id['Split'] == 'train'].values.astype('uint8')
            self.train_test_id = self.train_test_id[self.train_test_id['Split'] == 'train'].ID.values
            #print('Train =', self.train, 'train_test_id.shape: ', self.train_test_id.shape)
        elif train == 'valid':
            self.labels_ids = self.labels_ids[self.train_test_id['Split'] != 'train'].values.astype('uint8')
            self.train_test_id = self.train_test_id[self.train_test_id['Split'] != 'train'].ID.values
            #print('Train =', self.train, 'train_test_id.shape: ', self.train_test_id.shape)

        self.n = self.train_test_id.shape[0]

    def __len__(self):
        return self.n

    def transform_fn(self, image, mask):

        image = array_to_img(image, data_format="channels_last")
        mask_pil_array = [None] * mask.shape[-1]
        for i in range(mask.shape[-1]):
            mask_pil_array[i] = array_to_img(mask[:, :, i, np.newaxis], data_format="channels_last")
        if 'hflip' in self.augment_list:
            if random.random() > 0.5:
                image = TF.hflip(image)
                for i in range(mask.shape[-1]):
                    mask_pil_array[i] = TF.hflip(mask_pil_array[i])
        if 'vflip' in self.augment_list:
            if random.random() > 0.5:
                image = TF.vflip(image)
                for i in range(mask.shape[-1]):
                    mask_pil_array[i] = TF.vflip(mask_pil_array[i])
        if 'affine' in self.augment_list:
            angle = random.randint(0, 90)
            translate = (random.uniform(0, 100), random.uniform(0, 100))
            scale = random.uniform(0.5, 2)
            image = TF.affine(image, angle, translate, scale)
            for i in range(mask.shape[-1]):
                mask_pil_array[i] = TF.affine(mask_pil_array[i], angle, translate, scale)
        if 'adjust_brightness' in self.augment_list:
            if random.random() > 0.3:
                brightness_factor = random.uniform(0.8, 1.2)
                image = TF.adjust_brightness(image, brightness_factor)
        if 'adjust_saturation' in self.augment_list:
            if random.random() > 0.3:
                saturation_factor = random.uniform(0.8, 1.2)
                image = TF.adjust_saturation(image, saturation_factor)

        image = img_to_array(image, data_format="channels_last")
        if self.mask_use:
            for i in range(mask.shape[-1]):
                mask[:, :, i] = img_to_array(mask_pil_array[i], data_format="channels_last")[:, :, 0].astype('uint8')

        image = (image / 255.0).astype('float32')
        mask = (mask / 255.0).astype('uint8')

        if self.pretrained:
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image = (image - mean)/std

        return image, mask

    def __getitem__(self, index):

        name = self.train_test_id[index]
        path = self.image_path
        # Load image and from h5
        image = load_image(os.path.join(path, '%s.h5' % name), 'image')
        mask = np.empty([image.shape[0], image.shape[1], len(self.attribute)])
        if self.mask_use:
            for i, attr in enumerate(self.attribute):
                mask[:, :, i] = load_image(os.path.join(path, '{}_{}.h5'.format(name, attr)), 'mask')[:, :, 0]
        else:
            mask = image

        if self.train == 'train':
            if self.augment_list:
                image, mask = self.transform_fn(image, mask)
        """fig = plt.figure(figsize=(5, 5))
        ax = []
        for channel in range(mask.shape[2]):
            im = mask[:, :, channel]
            ax.append(fig.add_subplot(2, mask.shape[2], channel + 1))
            ax[channel].set_title(name[5:]+str(np.unique(im)))
            plt.imshow(im)
        print(np.unique(mask))"""
        if self.mask_use:
            p = np.random.uniform(0, 1)
            if self.train == 'valid':
                mask.fill(0.)
            elif self.cell:
                for i in range(0, image.shape[0], self.cell_size):
                    for j in range(0, image.shape[0], self.cell_size):
                        p = np.random.uniform(0, 1)
                        if p < self.prob:
                            mask[i:i+self.cell_size, j:j+self.cell_size, :].fill(0.)
            else:
                if p < self.prob:
                    mask.fill(0.)
            image_with_mask = np.dstack((image, mask))
        else:
            image_with_mask = image

        """for channel in range(mask.shape[2]):
            im = mask[:, :, channel]
            ax.append(fig.add_subplot(2, mask.shape[2], mask.shape[2] + channel + 1))
            ax[mask.shape[2] + channel].set_title(name[5:]+str(np.unique(im)))
            plt.imshow(im)
        plt.show()"""
        labels = np.array([self.labels_ids[index, ind] for ind in self.indexes])
        
        return np.array([image_with_mask for i in range(3)]), np.array([labels for i in range(3)]), name


def make_loader(train_test_id, labels_ids, args, train=True, shuffle=True):

    data_set = MyDataset(train_test_id=train_test_id,
                         labels_ids=labels_ids,
                         args=args,
                         train=train)
    data_loader = DataLoader(data_set,
                             batch_size=args.batch_size,
                             shuffle=shuffle,
                             num_workers=args.workers,
                             pin_memory=torch.cuda.is_available()
                             )
    return data_loader

Using TensorFlow backend.


In [3]:
class Args:
    
    def __init__(self):
        self.attribute = ['attribute_milia_like_cyst', 'attribute_pigment_network']
        self.pretrained = True
        self.show_model = True
        self.mask_use = True
        self.freezing = True
        self.prob = 0.1
        self.image_path = '/home/irek/PycharmProjects/Data/h5/'
        self.batch_size = 1
        self.model = 'resnet50'
        self.lr = 0.00001
        self.augment_list = []
        self.N = 0
        self.cell = True
        self.cell_size = 28
        self.workers = 1
device = 'cuda'

In [4]:
args = Args()
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')
mode = 'train'
loader = make_loader(train_test_id, mask_ind, args, train=mode, shuffle=True)    

In [5]:
from torchvision.models.resnet import Bottleneck, model_urls, _resnet, conv1x1, conv3x3
from torchvision.models.utils import load_state_dict_from_url
from torch.optim import Adam
from torch import nn

In [6]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        y = self.layer2(x)
        x = self.layer3(y)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x, y

    def forward(self, x):
        return self._forward_impl(x)
    
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [7]:
model = _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained=False, progress=True)
if args.mask_use:
    input_num = 3 + len(args.attribute)
    out_shape = len(args.attribute)
else:
    input_num = 3
    out_shape = len(args.attribute)
model.conv1 = torch.nn.Conv2d(input_num, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
last_layer_in_channels = model.fc.in_features
model.fc = torch.nn.Linear(last_layer_in_channels, out_shape)
model.to(device)
optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=0.01)

In [8]:
from sklearn.metrics import confusion_matrix


class Metrics:

    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0

    def update(self, y_true, y_pred):

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[1, 0]).ravel()
        self.tp += tp
        self.fp += fp
        self.fn += fn
        self.tn += tn

    def compute(self, loss, ep, epoch_time):
        acc = (self.tp + self.tn) / (self.tp + self.tn + self.fp + self.fn)  # TP + TN / (TP + TN + FP + FN)
        prec = self.tp / (self.tp + self.fp)  # TP / (TP + FP)
        rec = self.tp / (self.tp + self.fn)  # TP / (TP + FN)
        f1 = 2 * prec * rec / (rec + prec)  # 2 * PREC * REC / (PREC + REC)
        return {
                    'epoch': ep,
                    'loss': loss.item(),
                    'epoch_time': epoch_time,
                    'accuracy': acc,
                    'precision': prec,
                    'recall' : rec,
                    'f1_score': f1
                }

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0

In [9]:
#torch.cuda.empty_cache()

In [10]:
metric = Metrics()

In [26]:
nll_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([0.5, 1.0]).to(device))

for i, (image_batch, labels_batch, names) in enumerate(loader):
    #print(names)
    #print(image_batch[0].shape, labels_batch[0].shape, 'batch')
    aux_output, last_output = [0,0], [0,0]
    loss1 = 0
    loss2 = 0

    im = image_batch[0].permute(0, 3, 1, 2).to(device).type(torch.cuda.FloatTensor)
    lab = labels_batch[0].to(device).type(torch.cuda.FloatTensor)
    
    last_output, aux_output = model(im)
    
    if isinstance(args.attribute, str):
        labels_batch = torch.reshape(lab, (-1, 1))

    loss1 = nll_loss(last_output, lab)
    #print(aux_output.std(dim=0))
    l = aux_output.std(dim=0).data
    
    loss2 = torch.sum(l)
    #print(loss2, 'loss')
    
    if mode == 'train':
        optimizer.zero_grad()
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
    
        outputs = torch.sigmoid(last_output)

        outputs = np.around(outputs.data.cpu().numpy().ravel())
        labels_batch = labels_batch.data.cpu().numpy().ravel()
        metric.update(labels_batch, outputs)
    

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [9]:
nll_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([0.5, 1.0]).to(device))

def make_step(model, mode, train_test_id, mask_ind, args, device, optimizer, results, metric, epoch,
              scheduler):
    start_time = time.time()
    loader = make_loader(train_test_id, mask_ind, args, train=mode, shuffle=True)
    n = len(loader)
    if mode == 'valid':
        torch.set_grad_enabled(False)
    for i, (image_batch, labels_batch, names) in enumerate(loader):
        if i == n - 1:
            print(f'\r', end='')
        elif i < n - 3:
            print(f'\rBatch {i} / {n} ', end='')
        """if i < 5 :
            fig = plt.figure(figsize=(10, 10))
            ax = []
            for i, image in enumerate(image_batch):
                for channel in range(3, image.shape[2]):
                    im = image.cpu().numpy()[:, :, channel]
                    ax.append(fig.add_subplot(len(image_batch), image.shape[2], i*(image.shape[2]-3)+channel-3 + 1))
                    ax[i*(image.shape[2]-3)+channel-3].set_title(str(np.unique(im))) #names[i][5:]+
                    plt.imshow(im)
            plt.show()"""
        image_batch = image_batch[0].permute(0, 3, 1, 2).to(device).type(torch.cuda.FloatTensor)
        labels_batch = labels_batch[0].to(device).type(torch.cuda.FloatTensor)
        last_output, aux_output = model(image_batch)

        if isinstance(args.attribute, str):
            labels_batch = torch.reshape(labels_batch, (-1, 1))

        loss1 = nll_loss(last_output, labels_batch)
        #print(aux_output.std(dim=0))
        l = aux_output.std(dim=0).data

        loss2 = torch.sum(l)
        loss = loss1 + loss2
        if mode == 'train':
            optimizer.zero_grad()
                 
            loss.backward()
            optimizer.step()

        outputs = torch.sigmoid(last_output)

        outputs = np.around(outputs.data.cpu().numpy().ravel())
        labels_batch = labels_batch.data.cpu().numpy().ravel()
        metric.update(labels_batch, outputs)

    epoch_time = time.time() - start_time

    metrics = metric.compute(loss, epoch, epoch_time)

    if mode == 'valid':
        torch.set_grad_enabled(True)
        scheduler.step(loss)

    print('Epoch: {} Loss: {:.6f} Accuracy: {:.4f} Precision: {:.4f} Recall: {:.4f} F1: {:.4f} Time: {:.4f}'.format(
        metrics['epoch'],
        metrics['loss'],
        metrics['accuracy'],
        metrics['precision'],
        metrics['recall'],
        metrics['f1_score'],
        metrics['epoch_time']))

    metric.reset()

    return metrics, results


In [11]:
results = 0
ep = 0
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=10, verbose=True)
metrics = [0,0]
metric = Metrics()
for i, mode in enumerate(['train', 'valid']):

    metrics[i], results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                    args=args, device=device, optimizer=optimizer,
                                    results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.741835 Accuracy: 0.7121 Precision: 0.7386 Recall: 0.7709 F1: 0.7544 Time: 195.0346
Epoch: 0 Loss: 0.412552 Accuracy: 0.6717 Precision: 0.7005 Recall: 0.7664 F1: 0.7320 Time: 6.6408


In [3]:
tt = torch.Tensor([[[0, 1, 2],
                    [3, 4, 5],
                    [6, 7, 8],
                    [9,10,11]],
                   [[12,13,14],
                    [15,16,17],
                    [18,19,20],
                    [21,22,23]]])
tt.shape

torch.Size([2, 4, 3])

In [9]:
for i in range(2):
    print(tt[i])   # std

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
tensor([[12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.]])


In [11]:
for i in range(4):
    print(tt[:,i,:]) # binary

tensor([[ 0.,  1.,  2.],
        [12., 13., 14.]])
tensor([[ 3.,  4.,  5.],
        [15., 16., 17.]])
tensor([[ 6.,  7.,  8.],
        [18., 19., 20.]])
tensor([[ 9., 10., 11.],
        [21., 22., 23.]])


In [7]:
tt[0]

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])

In [18]:
a = tt.view((-1, tt.shape[1:]))

TypeError: view(): argument 'size' must be tuple of ints, but found element of type torch.Size at pos 2

In [27]:
tt.size([1:])

SyntaxError: invalid syntax (<ipython-input-27-f61dc055c0ce>, line 1)

In [30]:
a = tt.reshape((-1, *tt.shape[-1:]))

In [31]:
a

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.]])

In [15]:
tt.shape[-2:]

torch.Size([4, 3])

In [6]:
tt.view(-1, 3)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.]])

In [8]:
tt.view(-1, 3)[:4]

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])

In [ ]:
batch = (b, a, h, w, c)